In [1]:
import os
import numpy as np
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from PIL import Image

# Define directories
train_dir = "Data/train"
test_dir = "Data/test"

# Define image dimensions
img_width, img_height = 299, 299  # InceptionV3 expects input images to be 299x299

# Define number of classes
num_classes = 5  # Assuming 5 classes: Dandelion, Daisy, Sunflower, Rose, Tulip

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

# Load InceptionV3 model without top layers
base_model = InceptionV3(weights=None, include_top=False, input_shape=(img_width, img_height, 3))

# Add custom top layers for classification
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)

# Create new model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using Model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Save the model
model.save("inceptionv3_custom_model.h5")

# Predict images
def predict_image_class(image_path):
    # Load and preprocess the image
    img = Image.open(image_path)
    img = img.resize((img_width, img_height))
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    predictions = model.predict(img_array)

    # Decode predictions
    predicted_class_index = np.argmax(predictions)
    predicted_class = list(train_generator.class_indices.keys())[predicted_class_index]

    return predicted_class

Found 2494 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Epoch 1/10


2024-04-23 11:16:48.081591: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 

In [ ]:
def pred(i):
    test_image_path = "Data/test/Image_" + str(i) + ".jpg"
    predicted_class = predict_image_class(test_image_path)
    return(predicted_class)

import random
import cv2

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 50))

for i in range(1,101):
    x = random.choice(range(900))
    print(x)
    fig.add_subplot(20,5,i)
    img = cv2.cvtColor(cv2.imread("Data/test/Image_" + str(x) + ".jpg"),cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis("off")
    plt.title(pred(x), fontsize=10)
    

    

In [ ]:
model.save("VGG16.keras")